In [ ]:
# # header
import sys
sys.path.append("../")

%load_ext autoreload
%autoreload 2

In [ ]:
# # imports
import torch
import os
import argparse
from pprint import pformat
from collections import OrderedDict
from src.composer import COCOTokens, COCOAnimals, BG20k
from src.composer import PerceptualGrouping_COCO, Recognition_COCO, Search_COCO, SearchGrid_COCO
from src.conductor import AttentionTrain
from src.modelv2 import AttentionModel
from src.utils import plot_all, plot_loss_all
from src.utils import build_loaders, get_n_parameters
from prelude import get_device, startup_folders, load_dicts


In [ ]:
start_folder = r"../pretrained/coco_v2"
results_folder, logger = startup_folders(start_folder, name=f"exp_coco")
data_path = r"../data"

In [ ]:
model_params = load_dicts(start_folder, "model_params")
tasks = load_dicts(start_folder, "tasks")
train_params = load_dicts(start_folder, "train_params")
DeVice, num_workers, pin_memory = get_device()
print(f"model_params: {model_params}")
print(f"tasks: {tasks}")
print(f"train_params: {train_params}")

In [ ]:
tasks['Recognition']["composer"] = Recognition_COCO
tasks['PerceptualGrouping']["composer"] = PerceptualGrouping_COCO
tasks['Search']["composer"] = Search_COCO
tasks['SearchGrid']["composer"] = SearchGrid_COCO

tasks['Recognition']["datasets"] = []
tasks['PerceptualGrouping']["datasets"] = []
tasks['Search']["datasets"] = []
tasks['SearchGrid']["datasets"] = []

tasks['Recognition']["dataloaders"] = []
tasks['PerceptualGrouping']["dataloaders"] = []
tasks['Search']["dataloaders"] = []
tasks['SearchGrid']["dataloaders"] = []

tasks['Recognition']["loss_s"] = (slice(0, None, None), slice(0, None, None))
tasks['PerceptualGrouping']["loss_s"] = (slice(1, None, None), slice(1, None, None))
tasks['Search']["loss_s"] = (slice(1, None, None), slice(1, None, None))
tasks['SearchGrid']["loss_s"] = (slice(1, None, None), slice(1, None, None))

tasks['Recognition']["loss_s"] = ((-1, ), (-1, ))
tasks['PerceptualGrouping']["loss_s"] = ((-1, ), (-1, ))
tasks['Search']["loss_s"] = ((-1, ), (-1, ))
tasks['SearchGrid']["loss_s"] = ((-1, ), (-1, ))

In [ ]:
# datasets and dataloaders
coco_tokens = COCOTokens(directory=data_path, animals=True, split=0.9)
train_tks, valid_tks, test_tks = coco_tokens.get_tokens()
train_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=0, tokens=train_tks)
valid_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=1, tokens=valid_tks)
test_coco = COCOAnimals(in_dims=model_params["in_dims"], directory=data_path, kind=2, tokens=test_tks)
train_bg = BG20k(root=data_path, kind="train")
test_bg = valid_bg = BG20k(root=data_path, kind="test")
DeVice, num_workers, pin_memory = get_device()
for o in tasks:
    if tasks[o]["composer"] in (Recognition_COCO , SearchGrid_COCO):
        tasks[o]["datasets"].append(tasks[o]["composer"](train_coco, **tasks[o]["params"], bg_dataset=train_bg))
        tasks[o]["datasets"].append(tasks[o]["composer"](valid_coco, **tasks[o]["params"], bg_dataset=valid_bg))
        tasks[o]["datasets"].append(tasks[o]["composer"](test_coco, **tasks[o]["params"], bg_dataset=test_bg))
    else:
        tasks[o]["datasets"].append(tasks[o]["composer"](train_coco, **tasks[o]["params"]))
        tasks[o]["datasets"].append(tasks[o]["composer"](valid_coco, **tasks[o]["params"]))
        tasks[o]["datasets"].append(tasks[o]["composer"](test_coco, **tasks[o]["params"]))
    tasks[o]["datasets"][1].build_valid_test()
    tasks[o]["datasets"][2].build_valid_test()
    tasks[o]["dataloaders"] = build_loaders(tasks[o]["datasets"], batch_size=train_params["batch_size"], num_workers=num_workers, pin_memory=pin_memory, shuffle=False)
assert model_params["n_classes"] == train_coco.n_classes, f"Number of n_classes {model_params['n_classes']} and n_classes {train_coco.n_classes} must be equal!"
tasks["PerceptualGrouping"]["class_weights"] = train_coco.class_weights if hasattr(train_coco, "class_weights") else None
tasks["Recognition"]["class_weights"] = train_coco.class_weights if hasattr(train_coco, "class_weights") else None


In [ ]:
# model and optimizer...
model = AttentionModel(**model_params)
model_trainer = AttentionTrain(model, None, None, tasks, logger, results_folder)
model_dir = os.path.join(start_folder, "model" + ".pth")
assert os.path.exists(model_dir), "Could not find the model.pth in the given dir!"
model.load_state_dict(torch.load(model_dir, map_location=DeVice))

In [ ]:
# plotting
plot_all(10, model, tasks, results_folder, "_test", DeVice, logger, False, "test")


In [ ]:
# evaluating...
model_trainer.eval(DeVice, "test", False)
model_trainer.eval(DeVice, "valid", False)

In [ ]:
for a in model.modules():
    if isinstance(a, torch.nn.BatchNorm2d):
        print(a)
        print(a.running_mean.mean() if a.running_mean is not None else None, a.running_var.mean() if a.running_var is not None else None)
        if a.running_mean is None or a.running_var is None:
            a.running_mean, a.running_var = torch.zeros(a.num_features).to(DeVice), torch.ones(a.num_features).to(DeVice)
        a.train()
        a.track_running_stats = True
        a.train()

In [ ]:
# model.bmv = {}
# model.bmv_stuff = []
# model.bmv_i = {}

In [ ]:
model_trainer.track_stats(DeVice, 16)

In [ ]:
model.bmv_i

In [ ]:
for k, v in model.bmv.items():
    print(k)
    print(v[0].mean(), v[1].mean())

In [ ]:
for a in model.modules():
    if isinstance(a, torch.nn.BatchNorm2d):
        print(a)
        print(a.running_mean.mean(), a.running_var.mean())
        # if a.running_mean is None or a.running_var is None:
        #     a.running_mean, a.running_var = torch.zeros(a.num_features).to(device), torch.ones(a.num_features).to(device)
        # a.train()
        # a.track_running_stats = True
        # a.train()

In [ ]:
# evaluating...
model_trainer.eval(DeVice, "test", False)
model_trainer.eval(DeVice, "valid", False)


In [ ]:
1297/1558

In [ ]:
model_trainer.track_stats(DeVice, 16)

In [ ]:
# evaluating...
model_trainer.eval(DeVice, "test", False)
model_trainer.eval(DeVice, "valid", False)


In [ ]:
model_trainer.track_stats(DeVice, 16)

In [ ]:
# evaluating...
model_trainer.stop_tracking()
model_trainer.eval(DeVice, "test", False)
model_trainer.eval(DeVice, "valid", False)

In [ ]:
import foolbox
from src.resnet_ import ResNet
import time

class SimpModel(torch.nn.Module):
    def __init__(self,
                 model: torch.nn.Module,
                 n: int
                 ):
        super().__init__()
        self.model = model
        self.n = n

    def __call__(self, x: torch.Tensor):
        return self.forward(x)

    def forward(self, x: torch.Tensor):
        self.model.eval()
        if x.ndim == 4:
            x = x.unsqueeze(1)
        elif x.ndim == 5:
            assert x.size(1) == 1
        x = x.repeat(1, self.n, 1, 1, 1)
        _ = self.model.forward(x, 0)
        y, _ = self.model.for_forward(x[:, -1])
        return y


def adv_robustness(model_, dataloader_, device_, kind, epsilons_, preprocessing_ = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)):
    bounds = bounds = (0, 1)
    n = dataloader_.dataset.n_iter
    model = model_ if isinstance(model_, ResNet) else SimpModel(model_, n)
    model.to(device_)
    model.eval()
    fmodel = foolbox.PyTorchModel(model, bounds=bounds, device=device_, preprocessing=preprocessing_)
    if kind == "Linf":
        attack = foolbox.attacks.LinfDeepFoolAttack(steps=100, loss='crossentropy')
    elif kind == "L2":
        attack = foolbox.attacks.L2DeepFoolAttack(steps=100, loss='crossentropy')
    else:
        raise ValueError(f"Unknown kind of attack: {kind}")
    pre_acc = 0.0
    n_samples = 0
    epsilons_ = epsilons_.to(device_) if isinstance(epsilons_, torch.Tensor) else torch.tensor([epsilons_]).to(device_)
    are_adv = torch.zeros_like(epsilons_)
    are_adv = are_adv.to(device_)
    for composites, labels, *_ in dataloader_:
        epoch_t = time.time()
        k = composites.size(0)
        n_samples += k
        composites, labels = composites.to(device_), labels.to(device_)
        composites, labels = composites[:, 0], labels[:, 0]
        p_acc = foolbox.utils.accuracy(fmodel, composites, labels)
        pre_acc += (p_acc * k)
        print(f"Pre-accuracy: {p_acc:.2f}")
        raw, clipped, is_adv = attack(fmodel, composites, labels, epsilons=epsilons_)
        print(f"Adverserial robustness: {(1.0 - 1.0 * is_adv).sum()/k:.2f}")
        for i, eps in enumerate(epsilons_):
            are_adv[i] += (1.0 - 1.0 * is_adv).sum()
        print(f"Batch time: {time.time() - epoch_t:.1f} sec.")
        if n_samples > 1000:
            break
    return pre_acc/n_samples, are_adv/n_samples



In [ ]:
adv_robustness(model, tasks["Recognition"]["dataloaders"][2], DeVice, "Linf", 0.001, None)